In [91]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import cross_val_score
from  sklearn.model_selection import GridSearchCV, train_test_split
#from sklearn.metrics import classification_report
#from sklearn.ensemble import VotingClassifier, BaggingClassifier

In [92]:
train_df= pd.read_csv("data/iris dataset/iris_train.csv")
test_df= pd.read_csv("data/iris dataset/iris_test.csv")

In [93]:
print(train_df.shape, test_df.shape)

(75, 6) (75, 4)


In [94]:
train_df.head(10)

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2
5,5,virginica,6.5,5.8,3.0,2.2
6,6,setosa,4.3,1.1,3.0,0.1
7,7,versicolor,6.7,5.0,3.0,1.7
8,8,versicolor,6.8,4.8,2.8,1.4
9,9,versicolor,6.6,4.4,3.0,1.4


In [95]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 75 non-null     int64  
 1   species            75 non-null     object 
 2   sepal length (cm)  75 non-null     float64
 3   petal length (cm)  75 non-null     float64
 4   sepal width (cm)   75 non-null     float64
 5   petal width (cm)   75 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 3.6+ KB


In [96]:
train_df.describe()

,id,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
count,75.000000,75.000000,75.000000,75.000000,75.000000
mean,37.000000,6.044000,4.258667,2.958667,1.384000
std,21.794495,0.842653,1.615390,0.388376,0.697106
min,0.000000,4.300000,1.100000,2.000000,0.100000
25%,18.500000,5.550000,3.750000,2.800000,1.000000
50%,37.000000,6.100000,4.700000,3.000000,1.500000
75%,55.500000,6.600000,5.300000,3.200000,1.850000
max,74.000000,7.700000,6.900000,4.000000,2.500000


In [117]:
X_train_cat = train_df["species"]
X_train_num = train_df[["sepal length (cm)", "petal length (cm)"]]
y_train = train_df[["sepal width (cm)", "petal width (cm)"]]

In [118]:
X_test_cat = test_df["species"]
X_test_num = test_df[["sepal length (cm)", "petal length (cm)"]]

In [119]:
from sklearn.preprocessing import StandardScaler

In [120]:
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.fit_transform(X_test_num)

In [121]:
#train_df = pd.concat([train_df,pd.get_dummies(train_df["Embarked"])],axis=1)

In [122]:
X_train_cat = pd.get_dummies(X_train_cat)
X_test_cat = pd.get_dummies(X_test_cat)

In [123]:
X_train = pd.concat([X_train_cat,pd.DataFrame(X_train_num)],axis=1)
X_test = pd.concat([X_test_cat,pd.DataFrame(X_test_num)],axis=1)

 # 모델선택 및 학습부분 

In [161]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [136]:
def score(model):
    model.fit(X_train, y_train)
    model_pred = model.predict(X_train)
    lin_mse = mean_squared_error(model_pred, y_train)
    lin_rmse = np.sqrt(lin_mse)
    return lin_rmse

In [147]:
lasso = Lasso(alpha =0.0005)
score_lasso = score(lasso)

In [155]:
elastic = ElasticNet(alpha=0.0005, l1_ratio=.9)
score_elastic = score(elastic)

In [162]:
linear = LinearRegression()
score_linear = score(linear)

In [164]:
rand_forest = RandomForestRegressor()
score_rf = score(rand_forest)

In [172]:
model_scores = [score_elastic, score_lasso, score_linear, score_rf]

In [173]:
model_scores_mat = pd.DataFrame(model_scores, columns = ["model_scores"], index=["Elastic","Lasso","Linear","Random_forest"])

In [174]:
model_scores_mat

,model_scores
Elastic,0.228986
Lasso,0.228988
Linear,0.228973
Random_forest,0.126395


In [220]:
rf_params = [
{'n_estimators': [10, 25, 30], 'max_features': [3, 5, 10], 
 'max_depth': [10, 50,60,  None], 'bootstrap': [True, False]}
]
lasso_params=[
    {
        "alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }
]
elastic_params = [
    {
         "alpha": [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
        "l1_ratio" : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, ]
    }
]

In [221]:
def tuneHyperparameters(model, params):

    gridSearch = GridSearchCV(model, params, verbose=0, cv=10, scoring="neg_mean_absolute_error", n_jobs = -1)
    gridSearch.fit(X_train, y_train)
    bestParams, bestScore = gridSearch.best_params_,gridSearch.best_score_
    return bestScore, bestParams

In [222]:
model_names_to_tune = ["Random_forest","Lasso","Elastic"]
model_to_tune = [rand_forest, lasso, elastic]
model_params = [rf_params, lasso_params, elastic_params]
bestscores = list(map(tuneHyperparameters, model_to_tune, model_params))

C:\Users\hjhhi\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-0.23079211 -0.23185742 -0.22592316 -0.22879658 -0.22629405 -0.22627872
         nan         nan         nan -0.22408482 -0.23045196 -0.22825425
 -0.23674824 -0.22242815 -0.2234359          nan         nan         nan
 -0.23692872 -0.22902619 -0.22669743 -0.23130432 -0.23054292 -0.22758145
         nan         nan         nan -0.24045238 -0.22915565 -0.22932693
 -0.23490893 -0.22472887 -0.22976161         nan         nan         nan
 -0.25907515 -0.2569756  -0.25731811 -0.27599405 -0.27373512 -0.27166071
         nan         nan         nan -0.25601339 -0.25838036 -0.2613244
 -0.27446429 -0.27620357 -0.27499405         nan         nan         nan
 -0.25298214 -0.26500536 -0.25986161 -0.27898214 -0.27623571 -0.27605655
         nan         nan         nan -0.26231696 -0.25716429 -0.26371726
 -0.28075    -0.27651071 -0.27451786         nan       

In [223]:
bestscores

[(-0.22242815476190475,
  {'bootstrap': True, 'max_depth': 50, 'max_features': 5, 'n_estimators': 25}),
 (-0.1861414860398735, {'alpha': 0.001}),
 (-0.18587680880198199, {'alpha': 1e-05, 'l1_ratio': 1})]

In [227]:
estimator = ElasticNet(alpha=1e-05, l1_ratio=1)

In [229]:
estimator.fit(X_train, y_train)

ElasticNet(alpha=1e-05, l1_ratio=1)

In [230]:
final_submit = estimator.predict(X_test)

In [231]:
from sklearn.metrics import mean_absolute_error

In [241]:
submissionBagg = pd.DataFrame({"id":np.arange(0, 75) ,"sepal width (cm)": final_submit[:,0], "petal width (cm)":final_submit[:,1]})
submissionBagg.to_csv("example.csv", index = False)